<a href="https://colab.research.google.com/github/ccspen21/greenland-fishery-nowcast-2025/blob/main/model_run_reports.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!git clone https://github.com/ccspen21/greenland-fishery-nowcast-2025.git

fatal: destination path 'greenland-fishery-nowcast-2025' already exists and is not an empty directory.


In [4]:
!pip install requests xarray pandas pyjstat datetime pydap netCDF4
import os
import sqlite3
import pandas as pd
import requests
import time as time_module  # Rename to avoid shadowing
from pyjstat import pyjstat
from urllib.parse import quote
from io import StringIO
from IPython.display import display
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

# Ensure compatibility with Colab and GitHub
!apt-get update && apt-get install -y iputils-ping

# Define a configurable database path
DB_PATH = os.getenv("DB_PATH", "greenland_fishery.db")

# Helper function to validate DataFrame against schema
def validate_dataframe(df, expected_columns, dtypes):
    if df.empty:
        raise ValueError("DataFrame is empty, no rows found.")
    if not all(col in df.columns for col in expected_columns):
        raise ValueError(f"DataFrame missing expected columns: {expected_columns}")
    for col, dtype in dtypes.items():
        if col in df.columns:
            if dtype == int:
                df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)
            else:
                df[col] = df[col].astype(dtype)
    if df.isnull().any().any():
        raise ValueError(f"DataFrame contains NaN values: {df.head()}")

# Helper function for API calls with retries and exponential backoff
def fetch_with_retries(url, max_retries=3, timeout=60, method='get', json=None):
    for attempt in range(max_retries):
        try:
            if method == 'get':
                response = requests.get(url, timeout=timeout)
            else:
                response = requests.post(url, json=json, timeout=timeout)
            response.raise_for_status()
            return response
        except requests.exceptions.RequestException as e:
            print(f"Attempt {attempt+1} failed: {e}")
            if attempt + 1 == max_retries:
                raise
            time_module.sleep(2 ** attempt)  # Use time_module to avoid shadowing

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 257 kB in 1s (175 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package list

In [5]:
%cd /content/greenland-fishery-nowcast-2025

import sqlite3
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Path to the pre-populated database in Google Drive (adjust the path as needed)
DB_PATH = '/content/drive/MyDrive/greenland_fishery.db'  # Update this path based on where setup_dataset.ipynb saves the database

# Connect to the pre-populated SQLite database
conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()
print(f"Connected to SQLite database at {DB_PATH}")

# Load DataFrames from the database
df_clean = pd.read_sql_query("SELECT * FROM total_catch", conn)
df_foreign_clean = pd.read_sql_query("SELECT * FROM foreign_catch", conn)
df_fish_clean = pd.read_sql_query("SELECT * FROM fish_export", conn)
df_sst_west_clean = pd.read_sql_query("SELECT * FROM sst_west", conn)
df_sst_east_clean = pd.read_sql_query("SELECT * FROM sst_east", conn)
df_sst_south_clean = pd.read_sql_query("SELECT * FROM sst_south", conn)

# Validate loaded DataFrames
expected_columns = {
    'total_catch': ['Year', 'Quarter', 'Total_Catch', 'Unit'],
    'foreign_catch': ['Year', 'Quarter', 'Foreign_Catch', 'Unit'],
    'fish_export': ['Year', 'Quarter', 'Fish_Export_Value_Million_Kr', 'Unit'],
    'sst_west': ['Year', 'Quarter', 'Sea_Surface_Temp_C_West', 'Melt_Active_West', 'Melt_Index_West'],
    'sst_east': ['Year', 'Quarter', 'Sea_Surface_Temp_C_East', 'Melt_Active_East', 'Melt_Index_East'],
    'sst_south': ['Year', 'Quarter', 'Sea_Surface_Temp_C_South', 'Melt_Active_South', 'Melt_Index_South']
}

dtypes = {
    'Year': int,
    'Quarter': str,
    'Total_Catch': float,
    'Foreign_Catch': float,
    'Fish_Export_Value_Million_Kr': float,
    'Sea_Surface_Temp_C_West': float,
    'Melt_Active_West': int,
    'Melt_Index_West': float,
    'Sea_Surface_Temp_C_East': float,
    'Melt_Active_East': int,
    'Melt_Index_East': float,
    'Sea_Surface_Temp_C_South': float,
    'Melt_Active_South': int,
    'Melt_Index_South': float,
    'Unit': str
}

# Validate each DataFrame
validate_dataframe(df_clean, expected_columns['total_catch'], dtypes)
validate_dataframe(df_foreign_clean, expected_columns['foreign_catch'], dtypes)
validate_dataframe(df_fish_clean, expected_columns['fish_export'], dtypes)
validate_dataframe(df_sst_west_clean, expected_columns['sst_west'], dtypes)
validate_dataframe(df_sst_east_clean, expected_columns['sst_east'], dtypes)
validate_dataframe(df_sst_south_clean, expected_columns['sst_south'], dtypes)

print("✅ All DataFrames loaded and validated successfully")

/content/greenland-fishery-nowcast-2025
Mounted at /content/drive
Connected to SQLite database at /content/drive/MyDrive/greenland_fishery.db


DatabaseError: Execution failed on sql 'SELECT * FROM fish_export': no such table: fish_export

In [ ]:
# Standardize Year and Quarter columns across all DataFrames
def fix_keys(df):
    df["Year"] = df["Year"].astype(int)
    df["Quarter"] = df["Quarter"].astype(str).str.replace("quarter ", "Q").str.replace("Quarter ", "Q")
    return df

# Apply to all component DataFrames
df_clean = fix_keys(df_clean)
df_fish_clean = fix_keys(df_fish_clean)
df_foreign_clean = fix_keys(df_foreign_clean)
df_sst_west_clean = fix_keys(df_sst_west_clean)
df_sst_east_clean = fix_keys(df_sst_east_clean)
df_sst_south_clean = fix_keys(df_sst_south_clean)

print("✅ Standardized Year and Quarter columns across all DataFrames")

# Time Series of Fish Catch

In [ ]:
# Time Series of Fish Catch
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

df_clean["Time"] = df_clean["Year"].astype(str) + " " + df_clean["Quarter"]
df_clean["Time_Index"] = range(len(df_clean))

# Plot the time series
plt.figure(figsize=(12,6))
plt.plot(df_clean["Time_Index"], df_clean["Total_Catch"], marker='o', linestyle='-')

# Format x-axis: show only one label per year (use Q1 as the anchor)
plt.xticks(
    ticks=df_clean[df_clean["Quarter"] == "Q1"]["Time_Index"],
    labels=df_clean[df_clean["Quarter"] == "Q1"]["Year"],
    rotation=45
)

plt.xlabel("Year")
plt.ylabel("Total Catch (Tonnes)")
plt.title("Total Fish Catch Over Time")
plt.grid(True)
plt.tight_layout()
plt.savefig('total_fish_catch_over_time.png')

# Interaction Term

In [ ]:
# Create three-way interaction terms in each SST regional DataFrame
df_sst_west_clean["Melt_SST_Interaction_West"] = (
    df_sst_west_clean["Melt_Active_West"] *
    df_sst_west_clean["Melt_Index_West"] *
    df_sst_west_clean["Sea_Surface_Temp_C_West"]
)

df_sst_east_clean["Melt_SST_Interaction_East"] = (
    df_sst_east_clean["Melt_Active_East"] *
    df_sst_east_clean["Melt_Index_East"] *
    df_sst_east_clean["Sea_Surface_Temp_C_East"]
)

df_sst_south_clean["Melt_SST_Interaction_South"] = (
    df_sst_south_clean["Melt_Active_South"] *
    df_sst_south_clean["Melt_Index_South"] *
    df_sst_south_clean["Sea_Surface_Temp_C_South"]
)

# Merged Dataset

In [ ]:
# Merged Dataset
# Start fresh from df_clean
df_merged_with_interactions = df_clean.copy()

# Merge standard right-hand-side variables
df_merged_with_interactions = df_merged_with_interactions.merge(df_fish_clean, on=["Year", "Quarter"], how="inner")

# Merge SST interaction terms
df_merged_with_interactions = df_merged_with_interactions.merge(
    df_sst_west_clean[["Year", "Quarter", "Melt_SST_Interaction_West"]],
    on=["Year", "Quarter"], how="inner"
).merge(
    df_sst_east_clean[["Year", "Quarter", "Melt_SST_Interaction_East"]],
    on=["Year", "Quarter"], how="inner"
).merge(
    df_sst_south_clean[["Year", "Quarter", "Melt_SST_Interaction_South"]],
    on=["Year", "Quarter"], how="inner"
)

# Merge foreign catch
df_merged_with_interactions = df_merged_with_interactions.merge(
    df_foreign_clean.drop(columns=["Unit"]),
    on=["Year", "Quarter"], how="left"
)

# Drop unnecessary columns
df_merged_with_interactions = df_merged_with_interactions.drop(
    columns=[col for col in df_merged_with_interactions.columns if "Unit" in col], errors="ignore"
)
df_merged_with_interactions = df_merged_with_interactions.drop(columns=["Time", "Time_Index"], errors="ignore")

# Order
df_merged_with_interactions["Quarter"] = pd.Categorical(
    df_merged_with_interactions["Quarter"], categories=["Q1", "Q2", "Q3", "Q4"], ordered=True
)
df_merged_with_interactions = df_merged_with_interactions.sort_values(by=["Year", "Quarter"]).reset_index(drop=True)

print("✅ Final merged dataset shape:", df_merged_with_interactions.shape)
display(df_merged_with_interactions.head())

In [ ]:
# Create df_model_with_interactions with Specific Lagged Variables
# Step 1: Start with the merged dataset
# (The merging already happened in the "Merged Dataset" section)

# Step 2: Create only the specific lagged variables needed
df_merged_with_interactions["Total_Catch_lag4"] = df_merged_with_interactions["Total_Catch"].shift(4)
df_merged_with_interactions["Foreign_Catch_lag4"] = df_merged_with_interactions["Foreign_Catch"].shift(4)
df_merged_with_interactions["Fish_Export_Value_Million_Kr_lag2"] = df_merged_with_interactions["Fish_Export_Value_Million_Kr"].shift(2)
df_merged_with_interactions["Melt_SST_Interaction_West_lag4"] = df_merged_with_interactions["Melt_SST_Interaction_West"].shift(4)
df_merged_with_interactions["Melt_SST_Interaction_East_lag1"] = df_merged_with_interactions["Melt_SST_Interaction_East"].shift(1)

# Step 3: Define the modeling dataset with only the required features
features = [
    "Total_Catch_lag4",
    "Foreign_Catch_lag4",
    "Melt_SST_Interaction_West_lag4",
    "Melt_SST_Interaction_East_lag1",
    "Fish_Export_Value_Million_Kr_lag2"
]

# Drop rows with NA from lagging and select only the required columns
df_model_with_interactions = df_merged_with_interactions[
    ["Year", "Quarter", "Total_Catch"] + features
].dropna().reset_index(drop=True)

# Step 4: Define X and y for modeling
y = df_model_with_interactions["Total_Catch"]
X = df_model_with_interactions[features]

# Final shape check
print("✅ Clean setup with specific lags — X shape:", X.shape, "y shape:", y.shape)
print("✅ Features used:", X.columns.tolist())

# Summary Statistics

In [ ]:
# Step 1: Identify main (non-lagged) variables
main_vars = [
    col for col in df_merged_with_interactions.columns
    if not col.endswith("_lag1")
    and not col.endswith("_lag2")
    and not col.endswith("_lag3")
    and not col.endswith("_lag4")
    and col not in ["Year", "Quarter"]
]

# Step 2: Subset numeric columns only (optional)
main_df = df_merged_with_interactions[main_vars].select_dtypes(include="number")

# Step 3: Generate descriptive statistics
summary_main = main_df.describe().T
summary_main["missing"] = main_df.isna().sum()
summary_main["skew"] = main_df.skew()
summary_main["kurtosis"] = main_df.kurt()

# Round for cleaner presentation
summary_main = summary_main.round(2)

# Step 4: Display
print("📊 Descriptive Statistics (Main Variables Only):")
display(summary_main)

# Correlation Matrix

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

main_vars = [
    col for col in df_merged_with_interactions.columns
    if not col.endswith("_lag1") and not col.endswith("_lag2")
    and not col.endswith("_lag3") and not col.endswith("_lag4")
    and col not in ["Year", "Quarter"]
]

main_corr = df_merged_with_interactions[main_vars].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(main_corr, cmap="coolwarm", annot=True, fmt=".2f")
plt.title("Correlation Matrix of Main (Non-Lagged) Variables")
plt.tight_layout()
plt.savefig('correlation_matrix.png')

# Nowcasting Q1 2025

In [ ]:
# ✅ Use all available data up to Q4 2024
X_train_lasso_q1 = df_model_with_interactions[
    (df_model_with_interactions["Year"] < 2025)
][[
    "Total_Catch_lag4",
    "Melt_SST_Interaction_West_lag4",
    "Foreign_Catch_lag4",
    "Melt_SST_Interaction_East_lag1",
    "Fish_Export_Value_Million_Kr_lag2"
]]

y_train_lasso_q1 = df_model_with_interactions[
    (df_model_with_interactions["Year"] < 2025)
]["Total_Catch"]

# Step 2: Q1 2025 input for LASSO
X_nowcast_lasso_q1 = pd.DataFrame([{
    "Total_Catch_lag4": df_model_with_interactions.iloc[-4]["Total_Catch_lag4"],
    "Melt_SST_Interaction_West_lag4": df_model_with_interactions.iloc[-4]["Melt_SST_Interaction_West_lag4"],
    "Foreign_Catch_lag4": df_model_with_interactions.iloc[-4]["Foreign_Catch_lag4"],
    "Melt_SST_Interaction_East_lag1": df_model_with_interactions.iloc[-1]["Melt_SST_Interaction_East_lag1"],
    "Fish_Export_Value_Million_Kr_lag2": df_model_with_interactions.iloc[-4]["Fish_Export_Value_Million_Kr_lag2"]
}])

# Step 3: LASSO pipeline
from sklearn.linear_model import Lasso
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

lasso_pipeline_q1 = make_pipeline(
    StandardScaler(),
    Lasso(alpha=1.0, max_iter=10000)
)

lasso_pipeline_q1.fit(X_train_lasso_q1, y_train_lasso_q1)

# Step 4: Predict
y_pred_lasso_q1 = lasso_pipeline_q1.predict(X_nowcast_lasso_q1)[0]
print(f"📈 LASSO Nowcast for Q1 2025: {round(y_pred_lasso_q1):,.0f} tons")

# Graph of Nowcast

In [ ]:
import matplotlib.pyplot as plt

# Rebuild time axis if needed
df_clean["Time"] = df_clean["Year"].astype(str) + " " + df_clean["Quarter"]
df_clean["Time_Index"] = range(len(df_clean))

# Define the index and value for Q4 2024 (last actual) and Q1 2025 (nowcast)
last_actual_index = df_clean["Time_Index"].max()
last_actual_value = df_clean["Total_Catch"].iloc[-1]

nowcast_index = last_actual_index + 1
nowcast_value = y_pred_lasso_q1

# Plot historical series
plt.figure(figsize=(12, 6))
plt.plot(df_clean["Time_Index"], df_clean["Total_Catch"], marker='o', linestyle='-', label="Historical")

# Add red dot for nowcast
plt.plot(nowcast_index, nowcast_value, 'ro', label="LASSO Nowcast (Q1 2025)", markersize=8)

# Add red connecting line from Q4 2024 to Q1 2025
plt.plot([last_actual_index, nowcast_index], [last_actual_value, nowcast_value], color='red', linestyle='-', linewidth=2)

# Update x-ticks to include 2025
xticks = list(df_clean[df_clean["Quarter"] == "Q1"]["Time_Index"])
xticks_labels = list(df_clean[df_clean["Quarter"] == "Q1"]["Year"].astype(str))
xticks.append(nowcast_index)
xticks_labels.append("2025")

plt.xticks(ticks=xticks, labels=xticks_labels, rotation=45)

plt.text(
    nowcast_index + 0.2,          # shift right
    nowcast_value + 2000,         # shift upward
    f"{round(nowcast_value):,}",
    color="red",
    fontsize=9.5,
)

plt.axvspan(nowcast_index - 0.1, nowcast_index + 0.1, color='red', alpha=0.1)

plt.axvline(nowcast_index, color='gray', linestyle='--', linewidth=1)

# Labels and formatting
plt.xlabel("Year")
plt.ylabel("Total Catch (Tonnes)")
plt.title("Total Fish Catch Over Time with LASSO Nowcast")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.savefig('total_fish_catch_with_nowcast.png')

# Capelin Dummy - Example of Policy Intervention

Note on Capelin Dummy
The Capelin_Open dummy variable is introduced here to model the impact of policy interventions (e.g., fishery closures in 2019, 2020, 2024, and 2025). This variable is used specifically for nowcasting in this notebook and is not included in the LASSO model fitting performed in model_fitting_diagnostics.ipynb. This allows us to isolate the effect of policy shocks in our nowcast analysis.


In [ ]:
# Add Capelin_Open dummy variable to the existing df_model_with_interactions
df_model_capelin = df_model_with_interactions.copy()

# Add Capelin_Open dummy variable
df_model_capelin["Capelin_Open"] = df_model_capelin["Year"].apply(
    lambda x: 0 if x in [2019, 2020, 2024, 2025] else 1
)

# Define target and features (include Capelin_Open)
y_capelin = df_model_capelin["Total_Catch"]
X_capelin = df_model_capelin[
    [
        "Total_Catch_lag4",
        "Foreign_Catch_lag4",
        "Melt_SST_Interaction_West_lag4",
        "Melt_SST_Interaction_East_lag1",
        "Fish_Export_Value_Million_Kr_lag2",
        "Capelin_Open"
    ]
]

# Final check
print("✅ X_capelin shape:", X_capelin.shape)
print("✅ y_capelin shape:", y_capelin.shape)
print("✅ Final predictors:", X_capelin.columns.tolist())

# Nowcast with Capelin Dummy

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

# Step 1: Training data (up to Q4 2024)
X_train_q1_2025_binary = df_model_capelin[
    (df_model_capelin["Year"] < 2025)
][[
    "Total_Catch_lag4",
    "Foreign_Catch_lag4",
    "Melt_SST_Interaction_West_lag4",
    "Melt_SST_Interaction_East_lag1",
    "Fish_Export_Value_Million_Kr_lag2",
    "Capelin_Open"
]]
y_train_q1_2025_binary = df_model_capelin[
    (df_model_capelin["Year"] < 2025)
]["Total_Catch"]

print("✅ Training set size with Capelin dummy:", X_train_q1_2025_binary.shape)

# Step 2: Input row for Q1 2025
X_nowcast_q1_2025_binary = pd.DataFrame([{
    "Total_Catch_lag4": df_model_capelin.iloc[-4]["Total_Catch_lag4"],
    "Foreign_Catch_lag4": df_model_capelin.iloc[-4]["Foreign_Catch_lag4"],
    "Melt_SST_Interaction_West_lag4": df_model_capelin.iloc[-4]["Melt_SST_Interaction_West_lag4"],
    "Melt_SST_Interaction_East_lag1": df_model_capelin.iloc[-1]["Melt_SST_Interaction_East_lag1"],
    "Fish_Export_Value_Million_Kr_lag2": df_model_capelin.iloc[-4]["Fish_Export_Value_Million_Kr_lag2"],
    "Capelin_Open": 0  # manually set to 0 because 2025 is closed
}])

print("✅ Nowcast input for Q1 2025 with Capelin dummy:")
display(X_nowcast_q1_2025_binary)

# Step 3: Lasso model pipeline
lasso_pipeline_q1_binary = make_pipeline(
    StandardScaler(),
    Lasso(alpha=1.0, max_iter=50000, random_state=42)
)

# Step 4: Fit and predict
lasso_pipeline_q1_binary.fit(X_train_q1_2025_binary, y_train_q1_2025_binary)
y_pred_q1_2025_binary = lasso_pipeline_q1_binary.predict(X_nowcast_q1_2025_binary)[0]

# Step 5: Output result
print(f"📈 📊 Lasso Nowcast for Q1 2025 (with Capelin dummy): {round(y_pred_q1_2025_binary):,.0f} tons")

## Graph Comparison of Nowcasts

In [ ]:
import matplotlib.pyplot as plt

# Rebuild time axis if needed
df_clean["Time"] = df_clean["Year"].astype(str) + " " + df_clean["Quarter"]
df_clean["Time_Index"] = range(len(df_clean))

# Final observed point (Q4 2024)
last_actual_index = df_clean["Time_Index"].max()
last_actual_value = df_clean["Total_Catch"].iloc[-1]

# Prediction point (Q1 2025)
nowcast_index = last_actual_index + 1
nowcast_std = y_pred_lasso_q1                 # Standard LASSO (no Capelin)
nowcast_binary = y_pred_q1_2025_binary        # LASSO with Capelin dummy

# Plot historical data
plt.figure(figsize=(12, 6))
plt.plot(df_clean["Time_Index"], df_clean["Total_Catch"], marker='o', linestyle='-', label="Historical")

# Plot both nowcasts as dots
plt.plot(nowcast_index, nowcast_std, 'ro', label="LASSO Nowcast (No Capelin)", markersize=8)
plt.plot(nowcast_index, nowcast_binary, 'go', label="LASSO Nowcast (With Capelin)", markersize=8)

# Connect last actual to each forecast
plt.plot([last_actual_index, nowcast_index], [last_actual_value, nowcast_std],
         color='red', linestyle='-', linewidth=2)
plt.plot([last_actual_index, nowcast_index], [last_actual_value, nowcast_binary],
         color='green', linestyle='-', linewidth=2)

# Annotate both nowcast values
plt.text(nowcast_index + 0.2, nowcast_std + 2000,
         f"{round(nowcast_std):,}", color="red", fontsize=9.5)

plt.text(nowcast_index + 0.2, nowcast_binary + 2000,
         f"{round(nowcast_binary):,}", color="green", fontsize=9.5)

# Forecast area shading and vertical marker
plt.axvspan(nowcast_index - 0.1, nowcast_index + 0.1, color='gray', alpha=0.1)
plt.axvline(nowcast_index, color='gray', linestyle='--', linewidth=1)

# X-axis setup
xticks = list(df_clean[df_clean["Quarter"] == "Q1"]["Time_Index"])
xticks_labels = list(df_clean[df_clean["Quarter"] == "Q1"]["Year"].astype(str))
xticks.append(nowcast_index)
xticks_labels.append("2025")
plt.xticks(ticks=xticks, labels=xticks_labels, rotation=45)

# Labels and formatting
plt.xlabel("Year")
plt.ylabel("Total Catch (Tonnes)")
plt.title("Total Fish Catch Over Time: LASSO Nowcast With and Without Capelin Dummy")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.savefig('nowcast_comparison.png')

# Actual vs. Predicted Plot for Backtests

In [ ]:
# Actual vs. Predicted Plot for Backtests
import matplotlib.pyplot as plt
import pandas as pd

# Step 1: Load the backtest results from Google Drive
try:
    backtest_results = pd.read_csv('/content/drive/MyDrive/backtest_results_2024.csv')
except FileNotFoundError:
    print("Error: Backtest results file not found. Ensure model_fitting_diagnostics.ipynb has been run to generate backtest_results_2024.csv.")
    raise

# Step 2: Extract quarters, actual, and predicted values
quarters = backtest_results["Quarter"].tolist()
actual_values = backtest_results["Actual"].tolist()
predicted_values = backtest_results["Predicted"].tolist()

# Step 3: Calculate y-axis padding
all_values = actual_values + predicted_values
y_min = min(all_values) * 0.9  # 10% below
y_max = max(all_values) * 1.1  # 10% above

# Step 4: Plot
plt.figure(figsize=(10, 6))
plt.plot(quarters, actual_values, label="Actual", marker='o', linewidth=2)
plt.plot(quarters, predicted_values, label="LASSO Prediction", marker='o', linewidth=2)
plt.ylim(y_min, y_max)
plt.title("📊 Actual vs. Predicted Fish Catch (LASSO, Q1–Q4 2024)")
plt.xlabel("Quarter")
plt.ylabel("Total Catch (tons)")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.savefig('actual_vs_predicted.png')

In [ ]:
# Close the database connection
try:
    conn.close()
    print("Database connection closed.")
except Exception as e:
    print(f"Error closing database connection: {e}")